In [1]:
import os

files = [f for f in os.listdir(os.path.abspath('')) if f.endswith('.txt')]

for file in files:
    with open(file, 'r') as f:
        print(file)
        contents = f.readlines()
        print(contents[0:2])


HH-001_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD RESPONDENTS\n', 'Households\n']
HH-002_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH AND WITHOUT ELECTRICITY\n']
HH-003_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH AND WITHOUT RADIO\n']
HH-004_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH AND WITHOUT TELEVISION\n']
HH-005_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'BY TYPE AND WITH TELEVISION\n']
HH-006_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH TELEVISION BY TYPE OF TELEVISION SERVICE\n']
HH-006_2.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH TELEVISION BY TYPE OF TELEVISION SERVICE\n']
HH-007_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH AND WITHOUT FIXED TELEPHONE LINE\n']
HH-008_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH AND WITHOUT CELLPHONE\n']
HH-010_1.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH A COMPUTER BY TYPE\n']
HH-010_2.txt
['REGIONAL DISTRIBUTION OF HOUSEHOLD\n', 'WITH A COMPUTER BY TYPE\n']
HH-011_1.txt
['

#### Relevant for viz (data from DICT)

* HH-012_1.txt - Regional Distribution of Households With and Without Internet Access
* HH-004_1.txt - Regional Distribution of Households With and Without Television

In [2]:
import pandas as pd

data = pd.read_csv('HH-012_1.txt', delimiter='\t', skiprows=2)
data.head()

,With Internet Access Without Internet Access
0,"NCR 1,000,671 33.2 2,008,874 66.8 3,009,545"
1,"CAR 115,495 29.8 272,693 70.2 388,188"
2,"REGION! 224,947 18.0 1,025,390 82.0 1,250,337"
3,"REGION II 154,330 18.0 701,564 82.0 855,893"
4,"REGION III 663,095 23.5 2,160,105 76.5 2,823,200"


In [3]:
data_tv = pd.read_csv('HH-004_1.txt', delimiter='\t', skiprows=2)
data_tv.head()

,With Television Without Television
0,"NCR 2,829,350 94.0 180,195 6.0 3,009,545"
1,"CAR 315,885 81.4 72,303 18.6 388,188"
2,"REGION I 1,090,157 87.2 160,180 12.8 1,250,337"
3,"REGION II 713,038 83.3 142,855 16.7 855,893"
4,"REGION III 2,668,170 94.5 155,029 5.5 2,823,200"


In [4]:
df_clean=pd.DataFrame(columns=['Region','With Internet', 'Without Internet', 'Total'])

# data to clean df
df_clean['Total'] = data[data.columns[0]].apply(lambda x:x.split()[-1])
df_clean['Without Internet'] = data[data.columns[0]].apply(lambda x: x.split()[-3])
df_clean['With Internet'] = data[data.columns[0]].apply(lambda x: x.split()[-5])
df_clean['Region'] = data[data.columns[0]].apply(lambda x: x.split()[:-5])
df_clean['Region'] = df_clean['Region'].apply(lambda x: ' '.join(x))
df_clean['Region'][2] = 'REGION I'

df_clean['With Internet'][6] = 52140

In [5]:
# TV data to clean df
df_clean.insert(3, 'With television', data_tv[data_tv.columns[0]].apply(lambda x: x.split()[-5]))
df_clean.insert(4, 'Without television', data_tv[data_tv.columns[0]].apply(lambda x: x.split()[-3]))

df_clean['With television'][6] = 501164
df_clean['With television'][11] = 514945

In [6]:
# remove commas and convert to numeric values
df_clean.replace(',', '', regex=True, inplace=True)
cols_to_conv = df_clean.columns[1:]

for col in cols_to_conv:
    df_clean[col] = pd.to_numeric(df_clean[col]).astype('int32')
    
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Region              17 non-null     object
 1   With Internet       17 non-null     int32 
 2   Without Internet    17 non-null     int32 
 3   With television     17 non-null     int32 
 4   Without television  17 non-null     int32 
 5   Total               17 non-null     int32 
dtypes: int32(5), object(1)
memory usage: 604.0+ bytes


In [7]:
hh_with = df_clean['With Internet'].sum()
hh_without = df_clean['Without Internet'].sum()
hh_total = hh_with + hh_without

print('national ICT Household Survey 2019')
print('Households With Access to Internet: ', hh_with)
print('Households Without Access to Internet: ', hh_without)
print('Households Surveyed: ', hh_total)

print(f'No access to Internet: {100 * hh_without / hh_total:.2f}%')

national ICT Household Survey 2019
Households With Access to Internet:  4123882
Households Without Access to Internet:  19236479
Households Surveyed:  23360361
No access to Internet: 82.35%


In [8]:
with_tv = df_clean['With television'].sum()
without_tv = df_clean['Without television'].sum()
tv_total = with_tv + without_tv

print('Households with TV: ', with_tv)
print('Households without TV: ', without_tv)
print('Households Surveyed: ', tv_total)

print(f'No access to television: {100 * without_tv / tv_total:.2f}%')

Households with TV:  19330082
Households without TV:  4030879
Households Surveyed:  23360961
No access to television: 17.25%


In [9]:
df_clean.drop(columns=['Total'], inplace=True)
df_clean

,Region,With Internet,Without Internet,With television,Without television
0,NCR,1000671,2008874,2829350,180195
1,CAR,115495,272693,315885,72303
2,REGION I,224947,1025390,1090157,160180
3,REGION II,154330,701564,713038,142855
4,REGION III,663095,2160105,2668170,155029
5,REGION IV-A,844147,2642039,3217334,268852
6,REGION IV-B,52140,634776,501164,185752
7,REGION V,75063,955923,774635,256351
8,REGION VI,177260,1528295,1388172,317384
9,REGION VII,186621,1633431,1326302,493750


In [10]:
df_clean.to_excel('net_v_tv.xlsx', index=False)